# 相对动能

1. 读取历史数据。
2. 清洗数据。
3. 计算相对动能。
4. 用表格展示结果。

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

In [2]:
def read_binance_annual_ohlcv(year: int) -> pd.DataFrame:
    filepath = os.path.join("../data", f"binance_daily_ohlcv_{year}.csv")
    return pd.read_csv(filepath, index_col="timestamp", parse_dates=True)


years = [2022, 2023, 2024]
ohlcv = pd.concat((read_binance_annual_ohlcv(year) for year in years))

In [3]:
len(ohlcv.symbol.unique())

143

In [4]:
def momentum(prices: pd.Series, period: int = 365) -> pd.Series:
    """
    Momentum indicator based on Andreas F. Clenow’s book 'Stocks on the Move'
    
    Momentum is calculated by multiplying the annualized exponential regression slope by the R^2
    coefficient of the regression calculation.
    
    Args:
        prices (pd.Series): asset close prices
        period (int): days to compute annualized return
    
    Return:
        Series of (slope, r2, adjusted slope)
    """
    y = np.log(prices)
    x = np.arange(len(y))
    slope, _, rvalue, *_ = linregress(x, y)
    if slope >= 0:
        adjusted_slope = ((1 + slope) ** period) * (rvalue ** 2)
    else:
        adjusted_slope = ((1 + slope) ** period) * (1 - rvalue ** 2)
    return pd.Series({
        "slope": slope,
        "r2": rvalue ** 2,
        "adjusted_slope": adjusted_slope
    })

In [6]:
ohlcv

,open,high,low,close,volume,symbol
timestamp,,,,,,
2022-01-01,46216.9300,47954.6300,46208.3700,47722.6500,1.960446e+04,BTC/USDT
2022-01-02,47722.6600,47990.0000,46654.0000,47286.1800,1.834046e+04,BTC/USDT
2022-01-03,47286.1800,47570.0000,45696.0000,46446.1000,2.766208e+04,BTC/USDT
2022-01-04,46446.1000,47557.5400,45500.0000,45832.0100,3.549141e+04,BTC/USDT
2022-01-05,45832.0100,47070.0000,42500.0000,43451.1300,5.178412e+04,BTC/USDT
...,...,...,...,...,...,...
2024-04-05,0.2961,0.2989,0.2780,0.2936,9.776790e+06,BAT/USDT
2024-04-06,0.2936,0.2991,0.2918,0.2964,3.313721e+06,BAT/USDT
2024-04-07,0.2963,0.3030,0.2954,0.3023,5.018225e+06,BAT/USDT


In [9]:
# 参数
momentum_period = 100  # 动能指标的窗口期

# 将长格式转换为宽格式
ohlcv_wide = ohlcv.pivot(columns="symbol", values="close")

# 计算动能指标
mom = ohlcv_wide.tail(momentum_period).apply(momentum).transpose()

# 计算常用的持有期收益率
ret_30d = ohlcv_wide.pct_change(30, fill_method=None).iloc[-1]
ret_60d = ohlcv_wide.pct_change(60, fill_method=None).iloc[-1]
ret_90d = ohlcv_wide.pct_change(90, fill_method=None).iloc[-1]

# 合并数据
metrics = pd.concat({
    "Slope": mom["slope"],
    "R2": mom["r2"],
    "AdjustedSlope": mom["adjusted_slope"],
    "ROC(30d)": ret_30d,
    "ROC(60d)": ret_60d,
    "ROC(90d)": ret_90d,
}, axis=1)

In [10]:
def color_roc(value):
    if value > 0:
        return 'color: green'
    elif value < 0:
        return 'color: red'
    else:
        return ''

styled_metrics = (
    metrics
    .sort_values("AdjustedSlope", ascending=False)
    .head(20)
    .style
    .format({"Slope": "{:.2f}", "R2": "{:.1%}", "AdjustedSlope": "{:.2f}", "ROC(30d)": "{:.1%}", "ROC(60d)": "{:.1%}", "ROC(90d)": "{:.1%}"})
    .map(color_roc, subset=['ROC(30d)', 'ROC(60d)', 'ROC(90d)'])
)

styled_metrics

,Slope,R2,AdjustedSlope,ROC(30d),ROC(60d),ROC(90d)
symbol,,,,,,
OM/USDT,0.03,94.3%,28096.75,164.3%,155.0%,1079.5%
FLOKI/USDT,0.03,76.3%,20332.61,-24.2%,561.5%,554.8%
PEPE/USDT,0.03,73.5%,14581.79,-14.1%,626.7%,451.9%
CKB/USDT,0.03,87.5%,10499.42,36.1%,519.8%,808.5%
ARKM/USDT,0.02,79.5%,2955.10,-34.0%,255.5%,280.5%
FET/USDT,0.02,80.7%,1509.20,-3.1%,340.5%,273.6%
AGIX/USDT,0.02,83.5%,1378.60,-14.9%,267.3%,276.4%
AR/USDT,0.02,80.5%,1183.56,-19.8%,277.6%,258.8%
JASMY/USDT,0.02,78.1%,985.96,-1.0%,276.4%,272.8%
